# qBraid Quantum Jobs: AWS Devices + Amazon Braket

In [1]:
# enable qBraid Quantum Jobs
!qbraid jobs enable amazon_braket

Successfully enabled qBraid Quantum Jobs in the amazon_braket environment.
Every AWS job you run will now be submitted through the qBraid API, so no access keys/tokens are necessary. 

To disable, run: `qbraid jobs disable amazon_braket`


In [2]:
# list installed environments, check jobs keyword is green for Amazon Braket
!qbraid envs list

# installed environments:
#
qsharp                         /opt/.qbraid/environments/qsharp_b54crn
default                  jobs  /opt/.qbraid/environments/qbraid_000000
qiskit                   jobs  /home/jovyan/.qbraid/environments/qiskit_9y9siy
amazon_braket            jobs  /home/jovyan/.qbraid/environments/aws_braket_kwx6dl
intel                          /opt/.qbraid/environments/intel_zr7hfq
qbraid_sdk               jobs  /home/jovyan/.qbraid/environments/qbraid_sdk_9j9sjy



In [3]:
# check qBraid credits
!qbraid jobs get-credits

You have 1000.00 remaining qBraid credits.


In [1]:
import boto3
from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Circuit

# create the Amazon Braket circuit
bell = Circuit().h(0).cnot(0, 1)

print(bell)

T  : |0|1|
          
q0 : -H-C-
        | 
q1 : ---X-

T  : |0|1|


In [2]:
# choose the cloud-based on-demand simulator to run your circuit
device = AwsDevice("arn:aws:braket:::device/quantum-simulator/amazon/sv1")

## UNCOMMENT THE FOLLOWING LINE TO ACCESS ARIA:
# device = AwsDevice("arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1")

In [3]:
# execute the circuit
task = device.run(bell, shots=10)

In [5]:
# display the results
print(task.result().measurement_counts)

In [8]:
# list submitted jobs
!qbraid jobs list

Fri Apr 21 2023 15:57:10 GMT+0000 (Coordinated Universal Time) arn:aws:braket:::device/quantum-simulator/amazon/sv1 arn:aws:braket:us-east-1:592242689881:quantum-task/ca33820b-987b-42e1-bac5-8b382324fe12



In [9]:
# check remaining qBraid credits
!qbraid jobs get-credits

You have 999.60 remaining qBraid credits.


In [10]:
client = boto3.client("braket")

In [11]:
# retrieve job data from braket boto3
boto_data = client.search_quantum_tasks(
    filters=[{"name": "quantumTaskArn", "operator": "EQUAL", "values": [task.id]}],
)

job_data = boto_data["quantumTasks"][0]
created = job_data["createdAt"]
ended = job_data["endedAt"]
cost = float(job_data["tags"]["costEstimate"])

print(f"createdAt: {created}")
print(f"endedAt: {ended}")
print(f"costEstimate: {cost} credits ~ ${cost/100}")

createdAt: 2023-04-21 15:57:09.862000+00:00
endedAt: 2023-04-21 15:57:11.556000+00:00
costEstimate: 0.4000012 credits ~ $0.004000012


In [12]:
# disable qBraid Quantum Jobs
!qbraid jobs disable amazon_braket

Disable successful. You are now submitting quantum jobs with your own AWS credentials.

To re-enable, run: `qbraid jobs enable amazon_braket`
